In [ ]:
import common_utils
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import common_utils

# Approximately 35 meters
radius_in_degrees = 0.00035

# Rome LU raster file
lulc_raster_file = "../lu_rasters/ESRI_LULC_2023_Rome.tif"

# Ecostress LST points file
eco_stress_points_file = "../lst_centroids/ecostress_lst_rome_centroids.gpkg"
gdf = gpd.read_file(eco_stress_points_file)  # Read the .gpkg file

gdf.shape

In [ ]:

# Define LULC remap dictionary
lulc_remap = {
    'lulc_0': 'no_data',
    'lulc_1': 'water',
    'lulc_2': 'trees',
    'lulc_4': 'flooded_veg',
    'lulc_5': 'crop',
    'lulc_7': 'built_area',
    'lulc_8': 'bare_ground',
    'lulc_9': 'snow_or_ice',
    'lulc_10': 'clouds',
    'lulc_11': 'range_land'
}

gdf['lulc_proportions'] = common_utils.calculate_lulc_proportions(lulc_raster_file, gdf['geometry'], radius_in_degrees, lulc_remap)


In [ ]:
# Expand LULC proportions into separate columns
lu_profile_gdf = pd.DataFrame(gdf['lulc_proportions'].tolist())
final_df = gpd.GeoDataFrame(pd.concat([gdf.drop(columns=['lulc_proportions']), lu_profile_gdf], axis=1),
                            geometry='geometry', crs="EPSG:4326")

In [ ]:
# Summary statistics for all columns in the final GeoDataFrame
summary_stats = final_df.describe()

# Print the summary statistics
print(summary_stats)

In [ ]:
output_filepath = "../lu_profiles/rome_2023_landuse_profile_35m.pkl"

final_df.set_crs(epsg=4326, inplace=True)

# Save to file or use further
final_df.to_file(output_filepath, driver='GPKG')